In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
import os, sys
import pandas as pd
import numpy as np
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
from scipy import stats

#Step 1: web crawling

In [ ]:
#politifact
true_news=pd.read_csv('/content/drive/Shareddrives/MY Files/W IEEEmsc:resilience+fake news+Web scrapping/5-News/polififactt_true.csv')
#most_true_news=pd.read_csv('/content/drive/Shareddrives/MY Files/W IEEEmsc:resilience+fake news+Web scrapping/5-News/smc_polififactt_mostly_true.csv')
half_true_news=pd.read_csv('/content/drive/Shareddrives/MY Files/W IEEEmsc:resilience+fake news+Web scrapping/5-News/polififactt_half_true.csv')
#most_false_news=pd.read_csv('/content/drive/Shareddrives/MY Files/W IEEEmsc:resilience+fake news+Web scrapping/5-News/smc_polififactt_mostly_false.csv')
false_news=pd.read_excel('/content/drive/Shareddrives/MY Files/W IEEEmsc:resilience+fake news+Web scrapping/5-News/polififactt_fake.xlsx')
#pants_on_fire_news=pd.read_csv('/content/drive/Shareddrives/MY Files/W IEEEmsc:resilience+fake news+Web scrapping/5-News/smc_polififactt_pants_on_fire.csv')
#Snopos
Snopos_news=pd.read_excel('/content/drive/Shareddrives/MY Files/W IEEEmsc:resilience+fake news+Web scrapping/5-News/snopos_all_news1.xlsx')
#Poynter
Poynter_news=pd.read_csv('/content/drive/Shareddrives/MY Files/W IEEEmsc:resilience+fake news+Web scrapping/5-News/poynter_false_USA.csv')

https://matix.io/extract-text-from-webpage-using-beautifulsoup-and-python/

In [ ]:
!pip install interruptingcow

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
from interruptingcow import timeout


def get_text(url):
       res = requests.get(url)
       html_page = res.content
       soup = BeautifulSoup(html_page, 'html.parser')
       text = soup.find_all(text=True)
       set([t.parent.name for t in text])
       output = ''
       blacklist = [
           '[document]', ##Added by me
            'a',
            'footer',
               'div',
           'header',
           'strong',
        'style',
        'title',
        'ul',
        'li',
        'nav',  ##Added by me
          'noscript',
            'head',
           'html',
           'meta',
           'head', 
           'input',
           'script',# there may be more elements you don't want, such as "style", etc.
                 ]
       for t in text:
           #if t.parent.name not in blacklist:
           #if (t.parent.name in 'p') | (t.parent.name in 'a'):  
           if (t.parent.name in 'p'): 
               output += '{} '.format(t)   
       return output

def get_politi_text(true_news):
     true_news['first_text']=''; true_news['Second_text']='';true_news['Third_text']=''
     Size=true_news.shape[0]
     for i in range(true_news.shape[0]):  
       print(i,' from ',Size)    
       if type(true_news['first_source-href'][i])== str :  
          #start = time.time()
          #while time.time() < start + 5:
         try: 
          try:
              with timeout(5, exception=RuntimeError):
                true_news.at[i,'first_text']=get_text(true_news['first_source-href'][i])
          except RuntimeError:
              pass 
         except:
          pass                      
       if type(true_news['second_sourse-href'][i]) == str:  
         try: 
          try:
              with timeout(5, exception=RuntimeError):
          #start = time.time()
          #while time.time() < start + 5:
                true_news.at[i,'Second_text']=get_text(true_news['second_sourse-href'][i])
          except RuntimeError:
              pass
         except:
           pass             
       if type(true_news['third_source-href'][i]) == str:   
         try: 
          try:
              with timeout(5, exception=RuntimeError):
          #start = time.time()
          #while time.time() < start + 5:
                true_news.at[i,'Third_text']=get_text(true_news['third_source-href'][i])
          except RuntimeError:
              pass
         except:
           pass           
     return true_news

In [ ]:
true_news=get_politi_text(true_news)
true_news.to_excel("True_news.xlsx") 
half_true_news=get_politi_text(half_true_news)
half_true_news.to_excel("Mixed_news.xlsx") 
false_news=get_politi_text(false_news)
false_news.to_csv("false_news.csv") 

In [ ]:
Poynter_news['external_text']=''
for i in range(Poynter_news.shape[0]):  
  print(i,' from ',Poynter_news.shape[0]) 
  if type(Poynter_news['external link-href'][i])== str :   Poynter_news.at[i,'external_text']=get_text(Poynter_news['external link-href'][i])
Poynter_news.to_csv("Poynter_news_update.csv")   

#Cleaning and clasification

In [9]:
#politifact
true_news=pd.read_excel('/content/drive/Shareddrives/MY Files/W IEEEmsc:resilience+fake news+Web scrapping/6- Updated news/True_news.xlsx')
half_true_news=pd.read_excel('/content/drive/Shareddrives/MY Files/W IEEEmsc:resilience+fake news+Web scrapping/6- Updated news/Mixed_news.xlsx')
false_news=pd.read_excel('/content/drive/Shareddrives/MY Files/W IEEEmsc:resilience+fake news+Web scrapping/6- Updated news/false_news.xlsx')
#Snopos
Snopos_news=pd.read_excel('/content/drive/Shareddrives/MY Files/W IEEEmsc:resilience+fake news+Web scrapping/6- Updated news/snopos_all_news1.xlsx')
#Poynter
Poynter_news=pd.read_csv('/content/drive/Shareddrives/MY Files/W IEEEmsc:resilience+fake news+Web scrapping/6- Updated news/Poynter_news_update.csv')
#factcheck
factcheck=pd.read_csv('/content/drive/Shareddrives/MY Files/W IEEEmsc:resilience+fake news+Web scrapping/6- Updated news/factcheck.csv')

In [ ]:
half_true_news.columns

Index(['Unnamed: 0', 'web-scraper-order', 'web-scraper-start-url', 'tiltle',
       'date', 'Sourse', 'link', 'link-href', 'key_words', 'whole_text',
       'first_source', 'first_source-href', 'second_sourse',
       'second_sourse-href', 'third_source', 'third_source-href', 'first_text',
       'Second_text', 'Third_text'],
      dtype='object')

In [10]:
import re
def only_quat(X):
    Y=re.findall('"([^"]*)"', X)
    Z=''
    for i in range(len(Y)):
            Z=Y[i]+' '+Z
    return Z 

def Column_modifying(X,Y):
     X['only_qoutation']=''
     if Y== 'Snopos_news':
        for i in range(X.shape[0]):
          if type(X['text'][i])==str:
            X.at[i,'only_qoutation']= only_quat(X['text'][i])
     elif Y=='factcheck':
        for i in range(X.shape[0]):
          if type(X['full_text'][i])==str:
            X.at[i,'only_qoutation']= only_quat(X['full_text'][i])
     elif Y== 'Poynter_news':
        for i in range(X.shape[0]):
          if type(X['external_text'][i])==str:
            X.at[i,'only_qoutation']= only_quat(X['external_text'][i])
     else: 
        for i in range(X.shape[0]):
          if type(X['whole_text'][i])==str:
            X.at[i,'only_qoutation']= only_quat(X['whole_text'][i])     
     return X    

for i in range(factcheck.shape[0]): 
  if str(factcheck['full_text'][i])==str:
     factcheck.at[i,'full_text']=factcheck['full_text'][i].partition('Sources')[0]     

In [11]:
Snopos_news=Column_modifying(Snopos_news,'Snopos_news')
factcheck=Column_modifying(factcheck,'factcheck')
Poynter_news=Column_modifying(Poynter_news,'Poynter_news')
Snopos_news.to_excel("Snopos_news.xlsx") 
factcheck.to_excel("factcheck.xlsx") 
Poynter_news.to_csv("Poynter_news.csv") 

############politifact#####################################
true_news=Column_modifying(true_news,'true_news')
half_true_news=Column_modifying(half_true_news,'half_true_news')
false_news=Column_modifying(false_news,'false_news')

Text similiarity

In [ ]:
import nltk, string
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('punkt') # if necessary...

stemmer = nltk.stem.porter.PorterStemmer()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)

def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]

'''remove punctuation, lowercase, stem'''
def normalize(text):
    return stem_tokens(nltk.word_tokenize(text.lower().translate(remove_punctuation_map)))

vectorizer = TfidfVectorizer(tokenizer=normalize, stop_words='english')

def cosine_sim(text1, text2):
    tfidf = vectorizer.fit_transform([text1, text2])
    return ((tfidf * tfidf.T).A)[0,1]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
import nltk, string
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('punkt') # if necessary...

stemmer = nltk.stem.porter.PorterStemmer()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)

def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]

'''remove punctuation, lowercase, stem'''
def normalize(text):
    return stem_tokens(nltk.word_tokenize(text.lower().translate(remove_punctuation_map)))

vectorizer = TfidfVectorizer(tokenizer=normalize, stop_words='english')

def cosine_sim(text1, text2):
  if  type(text1)==str and  type(text2)==str: 
    print('text1',text1)
    print('text2',text2)
    tfidf = vectorizer.fit_transform([text1, text2])
    return ((tfidf * tfidf.T).A)[0,1]
  else: 
    return 0


def f_final_text(X):
   X['final_text']=''
   for i in range(X.shape[0]):
      print('i',i)
      Y1,Y2,Y3=0,0,0
      if type(X['only_qoutation'][i])==str:
         if type(X['first_text'][i])==str and   len(X['first_text'][i].replace('\n ','')) > 20:  print('Y1',X['first_text'][i]); Y1=cosine_sim(X['only_qoutation'][i], X['first_text'][i])
         if type(X['Second_text'][i])==str and  len(X['Second_text'][i].replace('\n ',''))> 20: print('Y2',X['Second_text'][i]); Y2=cosine_sim(X['only_qoutation'][i], X['Second_text'][i])
         if type(X['Third_text'][i])==str and   len(X['Third_text'][i].replace('\n ','')) > 20:  print('Y3',X['Third_text'][i]); Y3=cosine_sim(X['only_qoutation'][i], X['Third_text'][i])  
         z=max([Y1,Y2,Y3])
         if Y1==z and Y1 > 0.5:   X.at[i,'final_text']=X['first_text'][i]
         elif Y2==z and Y2 > 0.5:   X.at[i,'final_text']=X['Second_text'][i]
         elif Y3==z and Y3 > 0.5:   X.at[i,'final_text']=X['Third_text'][i]
         else: X.at[i,'final_text']=X['only_qoutation'][i]
      if type(X['only_qoutation'][i]) =='' and type(X['first_text'][i])==str:
         X.at[i,'final_text']=X['first_text'][i]           
   return X
false_news=f_final_text(false_news)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
i 0
Y1 Please wait a few minutes before you try again. Â© 2021 Instagram 
text1 data from studies in a relevant animal model. should address, The most important of these animal studies were highly reassuring, because they showed that vaccinated monkeys intentionally challenged with the COVID-19 virus were actually protected from disease and death, compared to unvaccinated monkeys that were vulnerable to infection, The truth is the exact opposite of the misinformation these rumors are trying to spread. Animal studies were indeed conducted with each of the COVID-19 vaccines in widespread use, even before large-scale human studies began, skipped all animal trials. COVID-19 Jab: They skipped all animal trials because all animals were dying and went directly to people, skipped all animal trials because all animals were dying. 
text2 Please wait a few minutes before you try again. Â

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'formerli', 'forti', 'ha', 'henc', 'hereaft', 'herebi', 'hi', 'howev', 'hundr', 'inde', 'latterli', 'mani', 'meanwhil', 'moreov', 'mostli', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'seriou', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'therebi', 'therefor', 'thi', 'thu', 'togeth', 'twelv', 'twenti', 'veri', 'wa', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev', 'whi', 'yourselv'] not in stop_words.
  'stop_wor

Streaming output truncated to the last 5000 lines.
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
text1 
text2 We canâ€™t seem to find the page you were looking for. Please try our search or A-Z index. 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
Y3 Misinformation isn't going away just because it's a new year. Support trusted, factual information with a tax deductible contribution to PolitiFact. 
 
 
Homemade face masks for sale blow in the wind outside of a business, Tuesday, May 12, 2020, in San Antonio. San Antonio continues to encourage face masks or coverings as well as social distancing in an an effort to fight the coronavirus pandemic. (AP)
 N95 masks offer the most protection from viral particles, health officials say, but they should be reserved for health care workers who are in direct contact with infected patients. Mask effectiveness varies, but claims that cloth masks provide 0% protection arenâ€™t accurate. More studies need to be done to examine 

ValueError: ignored

In [ ]:
half_true_news=f_final_text(half_true_news)
true_news=f_final_text(true_news)
false_news=f_final_text(false_news)
true_news.to_excel("True_news.xlsx") 
half_true_news.to_excel("Mixed_news.xlsx") 
false_news.to_csv("false_news.csv") 

ValueError: ignored

Latent similarity indexing 

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('stopwords')
from sklearn.decomposition import TruncatedSVD
# If nltk stop word is not downloaded
# nltk.download('stopwords')
from nltk.corpus import stopwords

# List of documents



a1 = half_true_news['first_text'][5] #"He is a good dog."
a2 = half_true_news['only_qoutation'][5]
a3 = half_true_news['Second_text'][5]
a4 = half_true_news['Third_text'][5]
#a5 = "I have brown cat and dog."

df = pd.DataFrame()
df["documents"] = [a1,a2,a3,a4]





# Preprocessing
df['clean_documents'] = df['documents'].str.replace("[^a-zA-Z#]", " ")
df['clean_documents'] = df['clean_documents'].fillna('').apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))
df['clean_documents'] = df['clean_documents'].fillna('').apply(lambda x: x.lower())

stop_words = stopwords.words('english')

# tokenization
tokenized_doc = df['clean_documents'].fillna('').apply(lambda x: x.split())

# remove stop-words
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])

# de-tokenization
detokenized_doc = []
for i in range(len(df)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)

df['clean_documents'] = detokenized_doc

# TF-IDF vector
vectorizer = TfidfVectorizer(stop_words='english', smooth_idf=True)
X = vectorizer.fit_transform(df['clean_documents'])

# SVD represent documents and terms in vectors 
svd_model = TruncatedSVD(n_components=3, algorithm='randomized', n_iter=100, random_state=122)
lsa = svd_model.fit_transform(X)

# Documents - Topic vector
pd.options.display.float_format = '{:,.16f}'.format
topic_encoded_df = pd.DataFrame(lsa, columns = ["topic_1","topic_2","topic_3"])
topic_encoded_df["documents"] = df['clean_documents']
display(topic_encoded_df[["documents", "topic_1","topic_2","topic_3"]])

# Features or words used as features 
dictionary = vectorizer.get_feature_names()

# Term-Topic matrix
encoding_matrix = pd.DataFrame(svd_model.components_, index = ["topic_1","topic_2","topic_3"], columns = (dictionary)).T

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,documents,topic_1,topic_2,topic_3
0,detected javascript disabled browser please en...,0.0576661933290463,0.9914644883854367,0.1064029434200257
1,leadership wvgovernor justice west virginia hi...,0.5849451920654256,-0.1108250527854356,0.7877398670311037
2,error occurred cloudy skies early partly cloud...,0.6888998258034338,0.0615089826525254,-0.4579575143951269
3,state reported least one confirmed case covid ...,0.7260203781845391,-0.0478236280369625,-0.2085804382111612


Inside quataion mark

In [ ]:
import re
X='jaber  is a good person "while we expect he obtaine more" but we think he can, and he will obtain, "Dr. Xu told he is the best of the best" yes this is correct'
Y=re.findall('"([^"]*)"', X)
Z=''
for i in range(len(Y)):
    Z=Y[i]+' '+Z
print(Z)

#Example

In [ ]:
#set([t.parent.name for t in text])

In [ ]:
import requests
from bs4 import BeautifulSoup
#url = 'https://www.troyhunt.com/the-773-million-record-collection-1-data-reach/'
url=Poynter_news['external link-href'][1021]
#url= \
# 'https://www.cdc.gov/coronavirus/2019-ncov/vaccines/recommendations/adolescents.html'
res = requests.get(url)
html_page = res.content
soup = BeautifulSoup(html_page, 'html.parser')
text = soup.find_all(text=True)

set([t.parent.name for t in text])
output = ''
blacklist = [   '[document]', ##Added by me
     'a',
     'footer',
        'div',
    'header',
    'strong',
 'style',
 'title',
 'ul',
 'li',
 'nav',  ##Added by me
   'noscript',
     'head',
    'html',
    'meta',
    'head', 
    'input',
    'script',
    # there may be more elements you don't want, such as "style", etc.
]


for t in text:
    if t.parent.name not in blacklist:
    #if (t.parent.name in 'p') | (t.parent.name in 'a'):  
    #if (t.parent.name in 'p') :    
        output += '{} '.format(t)

print(output)


 
 
 
 
 
 
 
 ENG 
 What Lawsuit did Robert F. Kennedy Jr. Win and was Mandatory Vaccination Abolished in the U.S? 
 20 April, 2021 Share: 
 Social media users are still actively sharing the   uploaded by TV host Madona Koidze on August 14, 2020, in which Robert F. Kennedy Jr., an American anti-vaccine advocate and environmental lawyer, who is the nephew of former U.S. president John F. Kennedy, speaks about vaccines during an American talk show  . Kennedy recollects failed attempts of creating vaccines against SARS-Cov, respiratory syncytial (sin-SISH-uhl) viruses and dengue fever, saying that vaccines against these diseases had caused “enhanced immune response.” He claims that the novel coronavirus vaccines may pose the same threat, noting that such doubts are further strengthened by the fact that Anthony Fauci, director of the U.S. National Institute of Allergy and Infectious Diseases, allowed vaccine manufacturers to skip animal trials and to go directly to human trials. Madona K